# Day 7: NLP & Alternative Data - Interview Questions & Week Review

## Week 19 Summary & Interview Preparation

**Date:** Week 19, Day 7

**Topics Covered This Week:**
- Text preprocessing and tokenization for financial documents
- Sentiment analysis using traditional ML and deep learning
- Named Entity Recognition (NER) for financial entities
- Topic modeling (LDA, NMF) on earnings calls and news
- Word embeddings (Word2Vec, GloVe, FinBERT)
- Transformer-based models for finance (BERT, FinBERT)
- Alternative data sources and alpha generation

---

## Interview Questions Overview

| # | Topic | Difficulty | Key Concepts |
|---|-------|------------|-------------|
| 1 | Text Preprocessing | Medium | Tokenization, Lemmatization, Financial Text |
| 2 | Sentiment Analysis | Medium | Lexicon vs ML approaches, Domain adaptation |
| 3 | Word Embeddings | Medium | Word2Vec, GloVe, Contextual embeddings |
| 4 | FinBERT & Transformers | Hard | Transfer learning, Fine-tuning |
| 5 | Topic Modeling | Medium | LDA, Coherence, Interpretability |
| 6 | NER for Finance | Medium | Entity extraction, Relation extraction |
| 7 | Event-Driven Strategies | Hard | News impact, Event detection |
| 8 | Alternative Data | Medium | Data sources, Alpha decay |
| 9 | NLP Feature Engineering | Hard | Text-to-signal pipeline |
| 10 | Production NLP Systems | Hard | Latency, Scalability, Monitoring |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# NLP Libraries
import re
from collections import Counter

print("Week 19 Day 7: NLP & Alternative Data Interview Review")
print("=" * 60)

---

## Question 1: Text Preprocessing for Financial Documents

### Interview Question:
**"Describe your approach to preprocessing financial text data (earnings calls, SEC filings, news). What unique challenges exist compared to general NLP?"**

### Key Discussion Points:
1. Financial-specific tokenization challenges
2. Handling numbers, percentages, and financial terms
3. Domain-specific stop words
4. Preserving semantic meaning (negations, comparatives)

In [ ]:
class FinancialTextPreprocessor:
    """
    Interview Answer: Financial Text Preprocessing Pipeline
    
    Key differences from general NLP:
    1. Numbers carry meaning (10% vs 5% matters)
    2. Financial jargon (EBITDA, YoY, QoQ)
    3. Negations are critical ("not expecting growth" vs "expecting growth")
    4. Comparative language matters ("better than", "worse than")
    """
    
    def __init__(self):
        # Financial-specific patterns
        self.financial_terms = {
            'ebitda', 'eps', 'p/e', 'yoy', 'qoq', 'mom', 'cagr',
            'roi', 'roe', 'roa', 'wacc', 'dcf', 'fcf', 'capex'
        }
        
        # Numbers to preserve with context
        self.number_patterns = [
            (r'\$[\d,]+\.?\d*[BMK]?', 'DOLLAR_AMOUNT'),
            (r'\d+\.?\d*%', 'PERCENTAGE'),
            (r'\d+\.?\d*x', 'MULTIPLE'),
            (r'Q[1-4]', 'QUARTER'),
            (r'FY\d{2,4}', 'FISCAL_YEAR'),
        ]
        
        # Negation handling
        self.negation_words = {'not', 'no', "n't", 'never', 'neither', 'without'}
        
    def normalize_numbers(self, text: str) -> str:
        """Replace numbers with semantic tokens while preserving meaning."""
        for pattern, token in self.number_patterns:
            text = re.sub(pattern, f' {token} ', text)
        return text
    
    def handle_negations(self, tokens: List[str]) -> List[str]:
        """
        Mark tokens following negations.
        'not good' -> 'not' 'NEG_good'
        """
        result = []
        negate_next = False
        
        for token in tokens:
            if token.lower() in self.negation_words:
                negate_next = True
                result.append(token)
            elif negate_next:
                result.append(f'NEG_{token}')
                negate_next = False
            else:
                result.append(token)
                
        return result
    
    def preprocess(self, text: str) -> List[str]:
        """Full preprocessing pipeline."""
        # 1. Lowercase (but preserve acronyms)
        text = self._smart_lowercase(text)
        
        # 2. Normalize numbers
        text = self.normalize_numbers(text)
        
        # 3. Basic tokenization
        tokens = text.split()
        
        # 4. Handle negations
        tokens = self.handle_negations(tokens)
        
        return tokens
    
    def _smart_lowercase(self, text: str) -> str:
        """Lowercase while preserving financial acronyms."""
        words = text.split()
        result = []
        for word in words:
            if word.upper() == word and len(word) <= 5:  # Likely acronym
                result.append(word)
            else:
                result.append(word.lower())
        return ' '.join(result)

# Demonstration
preprocessor = FinancialTextPreprocessor()

sample_text = """
Our Q3 EBITDA grew by 15% YoY to $2.5B. However, we are not expecting 
similar growth in Q4 due to increased CAPEX of $500M.
"""

tokens = preprocessor.preprocess(sample_text)
print("Original text:")
print(sample_text)
print("\nProcessed tokens:")
print(tokens)

### Answer Framework:

**Financial text preprocessing differs from general NLP in several key ways:**

1. **Number Handling**: Unlike general NLP where numbers are often removed, financial numbers carry critical meaning. We normalize them to semantic tokens while preserving context.

2. **Domain Vocabulary**: Financial acronyms (EBITDA, EPS, P/E) must be preserved. Standard lemmatizers may corrupt these.

3. **Negation Sensitivity**: In finance, "exceeding expectations" vs "not exceeding expectations" has opposite trading implications. We explicitly mark negation scope.

4. **Temporal References**: Q1/Q2/Q3/Q4, YoY, MoM are critical for time-series alignment.

5. **Comparative Language**: "Better than expected" vs "worse than expected" signals require special handling.

---

## Question 2: Sentiment Analysis Approaches

### Interview Question:
**"Compare lexicon-based vs machine learning approaches for financial sentiment analysis. When would you use each?"**

In [ ]:
class FinancialSentimentAnalyzer:
    """
    Interview Answer: Comparison of Sentiment Analysis Approaches
    
    1. Lexicon-Based (Loughran-McDonald)
       - Pros: Interpretable, no training needed, domain-specific
       - Cons: Misses context, limited vocabulary
       
    2. Machine Learning (Traditional)
       - Pros: Learns patterns, better accuracy
       - Cons: Needs labeled data, may overfit
       
    3. Deep Learning (FinBERT)
       - Pros: Contextual understanding, state-of-the-art
       - Cons: Computationally expensive, black box
    """
    
    def __init__(self):
        # Loughran-McDonald Financial Sentiment Lexicon (subset)
        self.positive_words = {
            'achieve', 'attain', 'benefit', 'better', 'breakthrough',
            'gain', 'improve', 'increase', 'opportunity', 'outperform',
            'positive', 'profit', 'progress', 'strength', 'success',
            'surpass', 'upturn', 'upgrade', 'growth', 'exceeded'
        }
        
        self.negative_words = {
            'adverse', 'against', 'decline', 'decrease', 'deficit',
            'deteriorate', 'difficulty', 'fail', 'failure', 'loss',
            'negative', 'risk', 'threat', 'unable', 'uncertain',
            'unfavorable', 'weak', 'worsen', 'downturn', 'missed'
        }
        
        self.uncertainty_words = {
            'approximately', 'assume', 'believe', 'could', 'depend',
            'estimate', 'expect', 'fluctuate', 'may', 'might',
            'possible', 'predict', 'probably', 'risk', 'uncertain'
        }
        
        self.litigious_words = {
            'alleged', 'attorney', 'claim', 'court', 'defendant',
            'lawsuit', 'legal', 'litigation', 'penalty', 'plaintiff'
        }
    
    def lexicon_sentiment(self, text: str) -> Dict[str, float]:
        """Loughran-McDonald lexicon-based sentiment."""
        words = text.lower().split()
        total_words = len(words)
        
        if total_words == 0:
            return {'positive': 0, 'negative': 0, 'uncertainty': 0, 'litigious': 0}
        
        scores = {
            'positive': sum(1 for w in words if w in self.positive_words) / total_words,
            'negative': sum(1 for w in words if w in self.negative_words) / total_words,
            'uncertainty': sum(1 for w in words if w in self.uncertainty_words) / total_words,
            'litigious': sum(1 for w in words if w in self.litigious_words) / total_words
        }
        
        # Net sentiment score
        scores['net_sentiment'] = scores['positive'] - scores['negative']
        
        return scores
    
    def compare_approaches(self, texts: List[str]) -> pd.DataFrame:
        """Compare lexicon sentiment across multiple texts."""
        results = []
        for text in texts:
            scores = self.lexicon_sentiment(text)
            scores['text_preview'] = text[:50] + '...'
            results.append(scores)
        return pd.DataFrame(results)

# Demonstration
analyzer = FinancialSentimentAnalyzer()

sample_texts = [
    "Strong earnings growth exceeded expectations with profit up 25% and positive outlook for next quarter.",
    "The company faces significant risk and uncertainty due to declining sales and potential litigation.",
    "We believe revenue may increase but there could be some difficulty in achieving targets."
]

results = analyzer.compare_approaches(sample_texts)
print("Lexicon-Based Sentiment Analysis Results:")
print(results.round(3).to_string())

In [ ]:
# Visualization of sentiment comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Approach comparison
approaches = ['Lexicon\n(L-M)', 'ML\n(Traditional)', 'Deep Learning\n(FinBERT)']
metrics = {
    'Accuracy': [0.65, 0.78, 0.89],
    'Interpretability': [0.95, 0.60, 0.30],
    'Speed': [0.95, 0.80, 0.40],
    'Context Understanding': [0.30, 0.55, 0.90]
}

x = np.arange(len(approaches))
width = 0.2

for i, (metric, values) in enumerate(metrics.items()):
    axes[0].bar(x + i*width, values, width, label=metric)

axes[0].set_xlabel('Approach')
axes[0].set_ylabel('Score')
axes[0].set_title('Sentiment Analysis Approaches Comparison')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(approaches)
axes[0].legend(loc='upper right')
axes[0].set_ylim(0, 1.1)

# Use case decision tree
use_cases = ['Real-time\nTrading', 'Research\nAnalysis', 'Risk\nMonitoring', 'Regulatory\nCompliance']
recommended = {
    'Lexicon': [0.3, 0.5, 0.8, 0.9],
    'ML': [0.6, 0.7, 0.6, 0.5],
    'FinBERT': [0.4, 0.9, 0.5, 0.3]
}

x2 = np.arange(len(use_cases))
for i, (approach, scores) in enumerate(recommended.items()):
    axes[1].bar(x2 + i*0.25, scores, 0.25, label=approach)

axes[1].set_xlabel('Use Case')
axes[1].set_ylabel('Recommendation Score')
axes[1].set_title('Recommended Approach by Use Case')
axes[1].set_xticks(x2 + 0.25)
axes[1].set_xticklabels(use_cases)
axes[1].legend()

plt.tight_layout()
plt.show()

### Answer Framework:

**Use Lexicon-Based (Loughran-McDonald) when:**
- Need interpretability for compliance/audit
- Low latency requirements
- Limited labeled training data
- Baseline sentiment tracking

**Use ML-Based when:**
- Have labeled training data
- Need moderate accuracy improvement
- Can accept some latency

**Use Deep Learning (FinBERT) when:**
- Maximum accuracy is critical
- Context understanding matters (negations, sarcasm)
- Batch processing acceptable
- Research/analysis (not real-time trading)

---

## Question 3: Word Embeddings for Finance

### Interview Question:
**"Explain the difference between Word2Vec, GloVe, and contextual embeddings. How would you train domain-specific embeddings for financial text?"**

In [ ]:
class EmbeddingComparison:
    """
    Interview Answer: Word Embeddings Comparison
    
    Static Embeddings:
    - Word2Vec: Predicts context (Skip-gram) or word (CBOW)
    - GloVe: Global co-occurrence matrix factorization
    - Both produce ONE vector per word regardless of context
    
    Contextual Embeddings:
    - ELMo: Bidirectional LSTM
    - BERT: Transformer-based, different vector per context
    - "Bank" in "river bank" vs "bank account" → different vectors
    
    Financial Domain Adaptation:
    1. Train Word2Vec on SEC filings corpus
    2. Fine-tune BERT on financial text → FinBERT
    3. Domain-specific vocabulary handling
    """
    
    @staticmethod
    def demonstrate_word2vec_concept():
        """
        Simplified Word2Vec training demonstration.
        
        Skip-gram: Given word, predict context
        CBOW: Given context, predict word
        """
        # Financial context windows
        financial_corpus = [
            "stock price increased after earnings beat",
            "revenue growth exceeded analyst expectations",
            "market volatility increased during crisis",
            "earnings beat drove stock rally",
            "profit margin improved quarter over quarter",
            "risk exposure reduced through hedging",
            "dividend yield attracted income investors",
            "acquisition synergies boosted earnings"
        ]
        
        # Build vocabulary
        vocab = set()
        for sentence in financial_corpus:
            vocab.update(sentence.split())
        
        word_to_idx = {word: i for i, word in enumerate(sorted(vocab))}
        
        # Create skip-gram training pairs (simplified)
        window_size = 2
        training_pairs = []
        
        for sentence in financial_corpus:
            words = sentence.split()
            for i, target in enumerate(words):
                context_start = max(0, i - window_size)
                context_end = min(len(words), i + window_size + 1)
                
                for j in range(context_start, context_end):
                    if i != j:
                        training_pairs.append((target, words[j]))
        
        return {
            'vocab_size': len(vocab),
            'training_pairs_sample': training_pairs[:10],
            'total_pairs': len(training_pairs)
        }
    
    @staticmethod
    def embedding_properties():
        """Compare embedding properties."""
        comparison = pd.DataFrame({
            'Property': [
                'Training Objective',
                'Context Handling',
                'Polysemy',
                'Training Data',
                'Computation',
                'Finance Use Case'
            ],
            'Word2Vec': [
                'Local context prediction',
                'Fixed window',
                'Single vector per word',
                'Can train on domain corpus',
                'Fast training',
                'Document similarity, clustering'
            ],
            'GloVe': [
                'Global co-occurrence',
                'Whole corpus statistics',
                'Single vector per word',
                'Pre-trained + fine-tune',
                'Moderate',
                'Analogy tasks, visualization'
            ],
            'BERT/FinBERT': [
                'Masked LM + NSP',
                'Full bidirectional',
                'Different vector per context',
                'Requires fine-tuning',
                'Computationally expensive',
                'Sentiment, classification, QA'
            ]
        })
        return comparison

# Demonstration
comparison = EmbeddingComparison()

print("Skip-gram Training Pairs Example:")
w2v_demo = comparison.demonstrate_word2vec_concept()
print(f"Vocabulary size: {w2v_demo['vocab_size']}")
print(f"Total training pairs: {w2v_demo['total_pairs']}")
print(f"\nSample pairs (target, context):")
for pair in w2v_demo['training_pairs_sample'][:5]:
    print(f"  {pair}")

print("\n" + "="*60)
print("\nEmbedding Comparison:")
print(comparison.embedding_properties().to_string())

### Answer Framework:

**Key Differences:**

| Aspect | Word2Vec | GloVe | BERT/FinBERT |
|--------|----------|-------|-------------|
| Type | Static | Static | Contextual |
| Training | Predict context | Matrix factorization | Masked LM |
| Polysemy | ❌ | ❌ | ✅ |
| Speed | Fast | Moderate | Slow |

**Domain Adaptation for Finance:**
1. **Corpus Collection**: SEC filings, earnings calls, analyst reports
2. **Pre-training**: Train Word2Vec/FastText on financial corpus
3. **Fine-tuning**: Start from pre-trained BERT, continue training on financial text
4. **Vocabulary Extension**: Add financial terms to tokenizer

---

## Question 4: FinBERT and Transformer Models

### Interview Question:
**"Explain how FinBERT works and when you would use it over simpler approaches. What are the trade-offs?"**

In [ ]:
class FinBERTExplainer:
    """
    Interview Answer: FinBERT for Financial NLP
    
    FinBERT Architecture:
    1. Pre-trained BERT base (110M parameters)
    2. Further pre-trained on financial corpus
    3. Fine-tuned on financial sentiment (PhraseBank dataset)
    
    Key Advantages:
    - Contextual understanding of financial language
    - Handles negations and complex sentences
    - Transfer learning from massive pre-training
    
    Trade-offs:
    - Computational cost (GPU required)
    - Latency (not suitable for HFT)
    - Less interpretable than lexicon methods
    """
    
    @staticmethod
    def transformer_architecture_overview():
        """Key components of transformer architecture."""
        components = {
            'Input Embedding': {
                'purpose': 'Convert tokens to vectors',
                'includes': ['Token embeddings', 'Position embeddings', 'Segment embeddings']
            },
            'Self-Attention': {
                'purpose': 'Weigh importance of each token to others',
                'formula': 'Attention(Q,K,V) = softmax(QK^T/√d_k)V',
                'key_insight': 'Captures long-range dependencies'
            },
            'Multi-Head Attention': {
                'purpose': 'Multiple attention patterns simultaneously',
                'heads': '12 in BERT-base',
                'key_insight': 'Different heads learn different relationships'
            },
            'Feed-Forward': {
                'purpose': 'Non-linear transformation per position',
                'architecture': 'Two linear layers with GELU activation'
            },
            '[CLS] Token': {
                'purpose': 'Aggregate representation for classification',
                'usage': 'Extract this for sentiment classification'
            }
        }
        return components
    
    @staticmethod
    def finbert_vs_bert_differences():
        """Key differences between BERT and FinBERT."""
        differences = pd.DataFrame({
            'Aspect': [
                'Pre-training Corpus',
                'Vocabulary',
                'Fine-tuning Data',
                'Output Classes',
                'Domain Terms'
            ],
            'BERT': [
                'Wikipedia + BookCorpus',
                'General (30K tokens)',
                'General sentiment',
                'Positive/Negative',
                'May not understand'
            ],
            'FinBERT': [
                '+ Reuters, SEC filings',
                'Extended with financial terms',
                'Financial PhraseBank',
                'Positive/Negative/Neutral',
                'Properly represented'
            ]
        })
        return differences
    
    @staticmethod
    def when_to_use_finbert():
        """Decision framework for using FinBERT."""
        decision_matrix = {
            'Use FinBERT': [
                'Complex financial documents',
                'Negation-heavy text',
                'Research/analysis (batch)',
                'Accuracy is paramount',
                'GPU resources available'
            ],
            'Use Simpler Approach': [
                'Real-time trading signals',
                'Simple sentiment monitoring',
                'Limited compute resources',
                'Need interpretability',
                'High volume, low latency'
            ]
        }
        return decision_matrix

explainer = FinBERTExplainer()

print("Transformer Architecture Components:")
print("=" * 60)
for component, details in explainer.transformer_architecture_overview().items():
    print(f"\n{component}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

print("\n" + "=" * 60)
print("\nBERT vs FinBERT:")
print(explainer.finbert_vs_bert_differences().to_string())

print("\n" + "=" * 60)
print("\nDecision Framework:")
for category, items in explainer.when_to_use_finbert().items():
    print(f"\n{category}:")
    for item in items:
        print(f"  • {item}")

---

## Question 5: Topic Modeling for Financial Documents

### Interview Question:
**"How would you apply topic modeling to earnings calls? What insights can you extract and how would you validate the topics?"**

In [ ]:
class TopicModelingFinance:
    """
    Interview Answer: Topic Modeling for Financial Documents
    
    Applications:
    1. Identify themes in earnings calls over time
    2. Track management focus areas
    3. Compare topic distributions across companies/sectors
    4. Event detection (new topics emerging)
    
    Methods:
    - LDA: Probabilistic, interpretable
    - NMF: Non-negative, faster
    - BERTopic: Neural, better coherence
    """
    
    def __init__(self, n_topics: int = 5):
        self.n_topics = n_topics
        
    def demonstrate_lda_concept(self, documents: List[str]) -> Dict:
        """
        Simplified LDA concept demonstration.
        
        LDA Assumptions:
        1. Documents are mixtures of topics
        2. Topics are distributions over words
        3. Each word assigned to one topic
        """
        # Build vocabulary
        all_words = []
        for doc in documents:
            all_words.extend(doc.lower().split())
        
        word_freq = Counter(all_words)
        
        # Simulated topic distributions (what LDA would learn)
        simulated_topics = {
            'Topic 1 - Growth': ['revenue', 'growth', 'increase', 'expansion', 'market'],
            'Topic 2 - Risk': ['risk', 'uncertainty', 'volatility', 'exposure', 'hedge'],
            'Topic 3 - Operations': ['cost', 'efficiency', 'margin', 'operations', 'productivity'],
            'Topic 4 - Strategy': ['acquisition', 'investment', 'strategic', 'partnership', 'innovation'],
            'Topic 5 - Guidance': ['expect', 'outlook', 'forecast', 'guidance', 'target']
        }
        
        # Document-topic distribution (simulated)
        doc_topics = []
        for doc in documents:
            words = set(doc.lower().split())
            topic_scores = {}
            for topic_name, topic_words in simulated_topics.items():
                overlap = len(words.intersection(topic_words))
                topic_scores[topic_name] = overlap
            doc_topics.append(topic_scores)
        
        return {
            'topics': simulated_topics,
            'doc_topics': doc_topics,
            'vocab_size': len(word_freq)
        }
    
    @staticmethod
    def topic_validation_metrics():
        """
        How to validate topic quality.
        
        1. Coherence Score: Do top words co-occur?
        2. Perplexity: How well model predicts held-out docs?
        3. Human Evaluation: Are topics interpretable?
        4. Downstream Task: Does it improve prediction?
        """
        metrics = {
            'Coherence (C_v)': {
                'range': '0 to 1 (higher better)',
                'interpretation': 'Semantic similarity of top words',
                'good_score': '> 0.5'
            },
            'Perplexity': {
                'range': 'Lower is better',
                'interpretation': 'Model fit on held-out data',
                'warning': 'Can decrease even with bad topics'
            },
            'Topic Diversity': {
                'range': '0 to 1',
                'interpretation': 'Unique words across topics',
                'good_score': '> 0.6'
            }
        }
        return metrics

# Demonstration
topic_modeler = TopicModelingFinance()

sample_earnings_excerpts = [
    "Our revenue growth exceeded expectations with strong market expansion in Asia.",
    "We are managing risk exposure through hedging strategies amid market volatility.",
    "Cost efficiency initiatives improved our operating margin significantly.",
    "The strategic acquisition will drive growth and create synergies.",
    "We expect continued growth and are raising our full-year guidance."
]

results = topic_modeler.demonstrate_lda_concept(sample_earnings_excerpts)

print("Discovered Topics:")
print("=" * 60)
for topic_name, words in results['topics'].items():
    print(f"\n{topic_name}:")
    print(f"  Top words: {', '.join(words)}")

print("\n" + "=" * 60)
print("\nValidation Metrics:")
for metric, details in topic_modeler.topic_validation_metrics().items():
    print(f"\n{metric}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

---

## Question 6: Named Entity Recognition for Finance

### Interview Question:
**"How would you build a Named Entity Recognition system for financial documents? What entities are most valuable for trading signals?"**

In [ ]:
class FinancialNER:
    """
    Interview Answer: Named Entity Recognition for Finance
    
    Financial Entity Types:
    1. ORG: Companies, exchanges, regulators
    2. MONEY: Dollar amounts, percentages
    3. DATE: Earnings dates, fiscal periods
    4. PERSON: Executives, analysts
    5. TICKER: Stock symbols
    6. METRIC: EPS, EBITDA, P/E
    7. EVENT: M&A, IPO, bankruptcy
    
    Trading Signal Value:
    - Link entities to time for event-driven trading
    - Track executive changes
    - Identify M&A targets
    - Extract numerical guidance
    """
    
    def __init__(self):
        # Pattern-based entity recognition (simplified)
        self.patterns = {
            'TICKER': r'\b[A-Z]{1,5}\b(?=\s|,|\.|$)',
            'MONEY': r'\$[\d,]+\.?\d*[BMK]?',
            'PERCENTAGE': r'\d+\.?\d*%',
            'DATE': r'(?:Q[1-4]|FY)\s?\d{2,4}|\d{1,2}/\d{1,2}/\d{2,4}',
            'METRIC': r'\b(?:EPS|EBITDA|P/E|ROE|ROA|CAGR)\b'
        }
        
        # Company name indicators
        self.org_suffixes = ['Inc', 'Corp', 'LLC', 'Ltd', 'Co', 'Company', 'Group']
        
    def extract_entities(self, text: str) -> Dict[str, List[str]]:
        """Extract financial entities using patterns."""
        entities = {}
        
        for entity_type, pattern in self.patterns.items():
            matches = re.findall(pattern, text)
            if matches:
                entities[entity_type] = matches
        
        # Extract company names
        org_pattern = r'([A-Z][a-z]+(?:\s[A-Z][a-z]+)*\s(?:' + '|'.join(self.org_suffixes) + r'))'
        orgs = re.findall(org_pattern, text)
        if orgs:
            entities['ORGANIZATION'] = orgs
            
        return entities
    
    def entity_linking(self, entities: Dict[str, List[str]]) -> Dict:
        """
        Link entities to knowledge base (simplified).
        
        In production:
        - Link tickers to company database
        - Normalize dates to timestamp
        - Link executives to roles
        """
        linked = {}
        
        # Example: normalize money amounts
        if 'MONEY' in entities:
            normalized = []
            for amount in entities['MONEY']:
                # Parse and normalize
                value = amount.replace('$', '').replace(',', '')
                if 'B' in value:
                    normalized.append(float(value.replace('B', '')) * 1e9)
                elif 'M' in value:
                    normalized.append(float(value.replace('M', '')) * 1e6)
                elif 'K' in value:
                    normalized.append(float(value.replace('K', '')) * 1e3)
                else:
                    try:
                        normalized.append(float(value))
                    except:
                        normalized.append(value)
            linked['MONEY_NORMALIZED'] = normalized
            
        return linked

# Demonstration
ner = FinancialNER()

sample_news = """
Apple Inc reported Q3 2024 EPS of $1.26, beating estimates by 5%. 
Revenue reached $85.8B, up 8% YoY. Microsoft Corp and GOOGL also 
reported strong results. The P/E ratio expanded to 28x as AAPL 
raised FY2025 guidance.
"""

entities = ner.extract_entities(sample_news)
linked = ner.entity_linking(entities)

print("Extracted Entities:")
print("=" * 60)
for entity_type, values in entities.items():
    print(f"\n{entity_type}:")
    for value in values:
        print(f"  • {value}")

print("\n" + "=" * 60)
print("\nLinked/Normalized:")
print(linked)

---

## Question 7: Event-Driven Trading with NLP

### Interview Question:
**"Design an event-driven trading system using NLP. How would you handle news latency and signal decay?"**

In [ ]:
class EventDrivenNLPSystem:
    """
    Interview Answer: Event-Driven Trading with NLP
    
    System Components:
    1. News Ingestion: Real-time feeds (Reuters, Bloomberg)
    2. Entity Extraction: Identify affected securities
    3. Event Classification: Earnings, M&A, regulatory, etc.
    4. Sentiment/Impact Scoring: Predict price direction
    5. Signal Generation: Trade decision
    6. Execution: Speed is critical
    
    Latency Handling:
    - Use fast models (not FinBERT for trading)
    - Pre-compute entity mappings
    - Cache model predictions
    
    Signal Decay:
    - Signals have half-life (minutes to hours)
    - Early movers capture most alpha
    - Model decay function and adjust sizing
    """
    
    def __init__(self):
        self.event_categories = [
            'EARNINGS', 'M&A', 'GUIDANCE', 'EXECUTIVE',
            'REGULATORY', 'PRODUCT', 'LEGAL', 'MACRO'
        ]
        
    def classify_event(self, headline: str) -> Tuple[str, float]:
        """Classify news event type (simplified)."""
        headline_lower = headline.lower()
        
        event_keywords = {
            'EARNINGS': ['earnings', 'eps', 'revenue', 'profit', 'quarter'],
            'M&A': ['acquire', 'merger', 'buyout', 'takeover', 'bid'],
            'GUIDANCE': ['guidance', 'outlook', 'forecast', 'expect', 'raise'],
            'EXECUTIVE': ['ceo', 'cfo', 'resign', 'appoint', 'executive'],
            'REGULATORY': ['sec', 'fda', 'approved', 'regulatory', 'fine'],
            'PRODUCT': ['launch', 'product', 'release', 'innovation'],
            'LEGAL': ['lawsuit', 'settlement', 'litigation', 'court'],
            'MACRO': ['fed', 'rates', 'inflation', 'gdp', 'unemployment']
        }
        
        scores = {}
        for event_type, keywords in event_keywords.items():
            score = sum(1 for kw in keywords if kw in headline_lower)
            scores[event_type] = score
        
        best_event = max(scores, key=scores.get)
        confidence = scores[best_event] / 5.0  # Normalize
        
        return best_event, min(confidence, 1.0)
    
    def calculate_signal_decay(self, 
                               time_since_event: float,
                               event_type: str) -> float:
        """
        Calculate signal strength decay over time.
        
        Different events have different decay rates:
        - Earnings: Fast decay (minutes)
        - M&A: Slow decay (hours/days)
        - Macro: Medium decay (hours)
        """
        # Half-life in minutes
        half_lives = {
            'EARNINGS': 10,
            'M&A': 120,
            'GUIDANCE': 30,
            'EXECUTIVE': 60,
            'REGULATORY': 45,
            'PRODUCT': 60,
            'LEGAL': 90,
            'MACRO': 60
        }
        
        half_life = half_lives.get(event_type, 60)
        
        # Exponential decay
        decay_factor = np.exp(-0.693 * time_since_event / half_life)
        
        return decay_factor
    
    def generate_signal(self,
                        headline: str,
                        sentiment_score: float,
                        time_since_event: float) -> Dict:
        """Generate trading signal from news event."""
        # Classify event
        event_type, event_confidence = self.classify_event(headline)
        
        # Calculate decay
        decay = self.calculate_signal_decay(time_since_event, event_type)
        
        # Adjusted signal strength
        raw_signal = sentiment_score * event_confidence
        adjusted_signal = raw_signal * decay
        
        # Position sizing based on signal strength
        if abs(adjusted_signal) > 0.5:
            position_size = 'FULL'
        elif abs(adjusted_signal) > 0.3:
            position_size = 'HALF'
        elif abs(adjusted_signal) > 0.1:
            position_size = 'QUARTER'
        else:
            position_size = 'NONE'
        
        return {
            'event_type': event_type,
            'event_confidence': event_confidence,
            'raw_signal': raw_signal,
            'decay_factor': decay,
            'adjusted_signal': adjusted_signal,
            'direction': 'LONG' if adjusted_signal > 0 else 'SHORT',
            'position_size': position_size
        }

# Demonstration
system = EventDrivenNLPSystem()

test_headlines = [
    ("Apple beats earnings expectations, raises guidance for Q4", 0.8, 2),
    ("Microsoft to acquire gaming company in $10B deal", 0.6, 30),
    ("Fed signals potential rate cut amid slowing inflation", 0.4, 15),
    ("CEO resigns amid accounting investigation", -0.9, 5)
]

print("Event-Driven Signal Generation:")
print("=" * 70)

for headline, sentiment, time_mins in test_headlines:
    signal = system.generate_signal(headline, sentiment, time_mins)
    print(f"\nHeadline: {headline[:50]}...")
    print(f"  Event Type: {signal['event_type']}")
    print(f"  Raw Signal: {signal['raw_signal']:.3f}")
    print(f"  Decay ({time_mins} min): {signal['decay_factor']:.3f}")
    print(f"  Adjusted Signal: {signal['adjusted_signal']:.3f}")
    print(f"  Action: {signal['direction']} - {signal['position_size']}")

In [ ]:
# Visualize signal decay
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Signal decay curves
time_range = np.linspace(0, 180, 100)  # 0 to 180 minutes
event_types = ['EARNINGS', 'M&A', 'GUIDANCE', 'MACRO']

for event_type in event_types:
    decay_values = [system.calculate_signal_decay(t, event_type) for t in time_range]
    axes[0].plot(time_range, decay_values, label=event_type, linewidth=2)

axes[0].set_xlabel('Time Since Event (minutes)')
axes[0].set_ylabel('Signal Strength')
axes[0].set_title('Signal Decay by Event Type')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='50% decay')

# Alpha capture timeline
alpha_timeline = {
    'HFT (< 1s)': 40,
    'Fast Quant (1-60s)': 30,
    'Quant (1-5min)': 15,
    'Discretionary (5-60min)': 10,
    'Slow (> 1hr)': 5
}

categories = list(alpha_timeline.keys())
values = list(alpha_timeline.values())
colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, len(categories)))

axes[1].barh(categories, values, color=colors)
axes[1].set_xlabel('% of Total Alpha Captured')
axes[1].set_title('Alpha Capture by Speed (News Events)')

for i, v in enumerate(values):
    axes[1].text(v + 1, i, f'{v}%', va='center')

plt.tight_layout()
plt.show()

---

## Question 8: Alternative Data Sources

### Interview Question:
**"What alternative data sources have you worked with? How do you evaluate the alpha potential and decay of alternative data?"**

In [ ]:
class AlternativeDataEvaluator:
    """
    Interview Answer: Alternative Data Evaluation Framework
    
    Alternative Data Categories:
    1. Sentiment: News, social media, reviews
    2. Geolocation: Foot traffic, satellite imagery
    3. Transaction: Credit card, web traffic
    4. Expert: Analyst estimates, insider trades
    5. Government: Patents, regulatory filings
    
    Evaluation Criteria:
    1. Coverage: What securities/sectors?
    2. History: How far back?
    3. Frequency: Real-time to monthly
    4. Exclusivity: How many users?
    5. Alpha: Predictive power
    6. Decay: How fast does signal degrade?
    """
    
    def __init__(self):
        self.data_sources = {
            'Social Media Sentiment': {
                'coverage': 'Broad (public companies)',
                'frequency': 'Real-time',
                'exclusivity': 'Low (widely available)',
                'alpha_potential': 0.6,
                'decay_months': 6,
                'cost': 'Medium',
                'effort': 'High (NLP pipeline)'
            },
            'Satellite Imagery': {
                'coverage': 'Limited (retail, oil, agriculture)',
                'frequency': 'Daily to weekly',
                'exclusivity': 'High',
                'alpha_potential': 0.8,
                'decay_months': 18,
                'cost': 'Very High',
                'effort': 'Very High (CV pipeline)'
            },
            'Credit Card Data': {
                'coverage': 'Consumer companies',
                'frequency': 'Daily',
                'exclusivity': 'Medium',
                'alpha_potential': 0.85,
                'decay_months': 12,
                'cost': 'High',
                'effort': 'Medium'
            },
            'Web Scraping': {
                'coverage': 'Depends on target',
                'frequency': 'Real-time possible',
                'exclusivity': 'Can be high',
                'alpha_potential': 0.7,
                'decay_months': 3,
                'cost': 'Low',
                'effort': 'Medium-High'
            },
            'SEC Filings NLP': {
                'coverage': 'All US public companies',
                'frequency': 'Quarterly/Event',
                'exclusivity': 'Low (public data)',
                'alpha_potential': 0.65,
                'decay_months': 24,
                'cost': 'Low',
                'effort': 'Medium'
            }
        }
    
    def evaluate_roi(self, data_source: str) -> Dict:
        """Calculate estimated ROI for data source."""
        if data_source not in self.data_sources:
            return None
        
        source = self.data_sources[data_source]
        
        # Cost mapping
        cost_map = {'Low': 50000, 'Medium': 200000, 'High': 500000, 'Very High': 1000000}
        effort_map = {'Low': 1, 'Medium': 2, 'Medium-High': 3, 'High': 4, 'Very High': 5}
        
        # Simplified ROI calculation
        alpha_value = source['alpha_potential'] * 1000000  # Assume $1M alpha at 100%
        cost = cost_map.get(source['cost'], 200000)
        decay_factor = source['decay_months'] / 24  # Normalize to 2 years
        effort = effort_map.get(source['effort'], 3)
        
        lifetime_value = alpha_value * decay_factor
        roi = (lifetime_value - cost) / cost
        time_to_value = effort * 3  # months
        
        return {
            'estimated_annual_alpha': alpha_value,
            'setup_cost': cost,
            'decay_adjusted_value': lifetime_value,
            'roi': roi,
            'time_to_production_months': time_to_value
        }
    
    def comparison_matrix(self) -> pd.DataFrame:
        """Generate comparison matrix of all sources."""
        data = []
        for name, attrs in self.data_sources.items():
            row = {'Source': name}
            row.update(attrs)
            roi = self.evaluate_roi(name)
            row['roi'] = roi['roi']
            data.append(row)
        
        return pd.DataFrame(data)

# Demonstration
evaluator = AlternativeDataEvaluator()

print("Alternative Data Source Evaluation:")
print("=" * 80)

comparison = evaluator.comparison_matrix()
print(comparison[['Source', 'alpha_potential', 'decay_months', 'cost', 'roi']].to_string())

print("\n" + "=" * 80)
print("\nDetailed ROI Analysis - Credit Card Data:")
roi_analysis = evaluator.evaluate_roi('Credit Card Data')
for key, value in roi_analysis.items():
    if isinstance(value, float):
        print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value:,}")

---

## Question 9: NLP Feature Engineering for Alpha

### Interview Question:
**"Walk me through your process of converting raw text data into features for a predictive model. What features have you found most predictive?"**

In [ ]:
class NLPFeatureEngineering:
    """
    Interview Answer: Text-to-Feature Pipeline for Alpha
    
    Feature Categories:
    1. Sentiment Features: Score, magnitude, uncertainty
    2. Linguistic Features: Complexity, tone shift
    3. Entity Features: Counts, types, relationships
    4. Temporal Features: Change vs history
    5. Cross-sectional: vs peers, vs market
    
    Most Predictive (in my experience):
    1. Sentiment CHANGE (not level)
    2. Uncertainty increase
    3. Executive tone shift
    4. Unusual word frequency
    5. Q&A session sentiment divergence
    """
    
    def extract_features(self, text: str, historical_texts: List[str] = None) -> Dict:
        """Extract comprehensive NLP features."""
        features = {}
        
        # Basic text statistics
        words = text.split()
        sentences = text.split('.')
        
        features['word_count'] = len(words)
        features['avg_word_length'] = np.mean([len(w) for w in words])
        features['sentence_count'] = len(sentences)
        features['avg_sentence_length'] = len(words) / max(len(sentences), 1)
        
        # Readability (simplified Flesch-Kincaid)
        syllables = sum(self._count_syllables(w) for w in words)
        features['readability'] = 206.835 - 1.015 * (len(words) / max(len(sentences), 1)) - 84.6 * (syllables / max(len(words), 1))
        
        # Sentiment features
        positive_words = ['growth', 'increase', 'profit', 'success', 'strong', 'exceeded', 'positive']
        negative_words = ['decline', 'decrease', 'loss', 'risk', 'weak', 'missed', 'negative']
        uncertain_words = ['may', 'might', 'could', 'expect', 'believe', 'estimate', 'uncertain']
        
        text_lower = text.lower()
        features['positive_ratio'] = sum(1 for w in positive_words if w in text_lower) / len(words)
        features['negative_ratio'] = sum(1 for w in negative_words if w in text_lower) / len(words)
        features['uncertainty_ratio'] = sum(1 for w in uncertain_words if w in text_lower) / len(words)
        features['net_sentiment'] = features['positive_ratio'] - features['negative_ratio']
        
        # Numerical mentions (often predictive)
        features['number_count'] = len(re.findall(r'\d+\.?\d*%?', text))
        features['dollar_count'] = len(re.findall(r'\$[\d,]+', text))
        
        # Forward-looking language (critical for guidance)
        forward_words = ['expect', 'anticipate', 'outlook', 'guidance', 'forecast', 'project']
        features['forward_looking_ratio'] = sum(1 for w in forward_words if w in text_lower) / len(words)
        
        # Calculate changes if historical data available
        if historical_texts:
            historical_features = self.extract_features(historical_texts[-1])
            features['sentiment_change'] = features['net_sentiment'] - historical_features.get('net_sentiment', 0)
            features['uncertainty_change'] = features['uncertainty_ratio'] - historical_features.get('uncertainty_ratio', 0)
        
        return features
    
    def _count_syllables(self, word: str) -> int:
        """Simple syllable counter."""
        vowels = 'aeiou'
        count = 0
        prev_is_vowel = False
        for char in word.lower():
            is_vowel = char in vowels
            if is_vowel and not prev_is_vowel:
                count += 1
            prev_is_vowel = is_vowel
        return max(count, 1)
    
    @staticmethod
    def feature_importance_analysis():
        """Typical feature importance from earnings call prediction."""
        importance = {
            'sentiment_change': 0.18,
            'uncertainty_change': 0.15,
            'forward_looking_ratio': 0.12,
            'net_sentiment': 0.10,
            'number_count': 0.09,
            'readability_change': 0.08,
            'negative_ratio': 0.07,
            'positive_ratio': 0.06,
            'word_count_change': 0.05,
            'dollar_mentions': 0.05,
            'other': 0.05
        }
        return importance

# Demonstration
fe = NLPFeatureEngineering()

sample_earnings = """
We are pleased to report strong quarterly results with revenue growth of 15% year-over-year.
Our operating margin improved to 22%, exceeding expectations. Looking ahead, we expect 
continued momentum and are raising our full-year guidance. However, we remain cautious about
potential macroeconomic headwinds and may adjust our investment timeline if conditions change.
"""

features = fe.extract_features(sample_earnings)

print("Extracted NLP Features:")
print("=" * 60)
for feature, value in features.items():
    if isinstance(value, float):
        print(f"  {feature}: {value:.4f}")
    else:
        print(f"  {feature}: {value}")

print("\n" + "=" * 60)
print("\nTypical Feature Importance (Earnings Prediction):")
importance = fe.feature_importance_analysis()
for feat, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True):
    print(f"  {feat}: {imp:.0%}")

---

## Question 10: Production NLP Systems

### Interview Question:
**"How would you deploy an NLP-based trading signal in production? What are the key monitoring and validation considerations?"**

In [ ]:
class ProductionNLPSystem:
    """
    Interview Answer: Production NLP System Architecture
    
    Key Components:
    1. Data Ingestion: Real-time feeds, deduplication
    2. Processing Pipeline: Async, fault-tolerant
    3. Model Serving: Low latency, A/B testing
    4. Signal Generation: Aggregation, filtering
    5. Monitoring: Drift detection, quality metrics
    6. Alerting: Anomaly detection, circuit breakers
    
    Critical Considerations:
    - Latency budget by use case
    - Model staleness and retraining
    - Data quality monitoring
    - Position risk from NLP errors
    """
    
    def __init__(self):
        self.latency_budget_ms = 100  # Target latency
        self.model_version = '1.0'
        self.metrics_history = []
        
    def system_architecture(self) -> Dict:
        """Production system architecture."""
        architecture = {
            'Data Layer': {
                'News Feeds': ['Reuters', 'Bloomberg', 'SEC EDGAR'],
                'Message Queue': 'Kafka',
                'Deduplication': 'Redis',
                'Storage': 'TimescaleDB'
            },
            'Processing Layer': {
                'Orchestration': 'Kubernetes',
                'NLP Service': 'FastAPI + async',
                'Model Serving': 'TorchServe / TF Serving',
                'Caching': 'Redis (entity lookups)'
            },
            'Signal Layer': {
                'Aggregation': 'Time-weighted sentiment',
                'Filtering': 'Confidence threshold',
                'Output': 'Signal API / PMS integration'
            },
            'Monitoring Layer': {
                'Metrics': 'Prometheus + Grafana',
                'Logging': 'ELK Stack',
                'Alerting': 'PagerDuty',
                'Drift Detection': 'Custom ML monitors'
            }
        }
        return architecture
    
    def monitoring_metrics(self) -> Dict:
        """Key metrics to monitor in production."""
        metrics = {
            'Operational': {
                'latency_p50': 'Target < 50ms',
                'latency_p99': 'Target < 200ms',
                'throughput': 'Messages/second',
                'error_rate': 'Target < 0.1%',
                'queue_depth': 'Backlog indicator'
            },
            'Model Quality': {
                'sentiment_distribution': 'Detect drift from baseline',
                'confidence_distribution': 'Monitor uncertainty',
                'entity_coverage': '% tickers matched',
                'prediction_vs_actual': 'Rolling accuracy'
            },
            'Business': {
                'signal_count': 'Daily signals generated',
                'signal_hit_rate': 'Correct direction %',
                'pnl_attribution': 'P&L from NLP signals',
                'decay_rate': 'Signal degradation over time'
            }
        }
        return metrics
    
    def drift_detection(self, current_dist: Dict, baseline_dist: Dict) -> Dict:
        """
        Detect distribution drift in sentiment scores.
        
        Methods:
        - KL Divergence
        - Population Stability Index (PSI)
        - Kolmogorov-Smirnov test
        """
        # Simplified PSI calculation
        psi = 0
        for bucket in current_dist:
            if bucket in baseline_dist:
                curr = current_dist[bucket]
                base = baseline_dist[bucket]
                if curr > 0 and base > 0:
                    psi += (curr - base) * np.log(curr / base)
        
        # PSI interpretation
        if psi < 0.1:
            status = 'STABLE'
        elif psi < 0.25:
            status = 'WARNING'
        else:
            status = 'CRITICAL'
        
        return {
            'psi': psi,
            'status': status,
            'action': 'Investigate model' if status != 'STABLE' else 'None'
        }
    
    def failsafe_rules(self) -> List[Dict]:
        """Circuit breaker rules for production safety."""
        rules = [
            {
                'rule': 'Latency Circuit Breaker',
                'trigger': 'p99 > 500ms for 5 minutes',
                'action': 'Switch to fallback model'
            },
            {
                'rule': 'Error Rate Circuit Breaker',
                'trigger': 'Error rate > 5%',
                'action': 'Halt signal generation'
            },
            {
                'rule': 'Drift Alert',
                'trigger': 'PSI > 0.25',
                'action': 'Page on-call, reduce position sizes'
            },
            {
                'rule': 'Anomaly Detection',
                'trigger': 'Sentiment outside 3-sigma',
                'action': 'Manual review before trading'
            },
            {
                'rule': 'Data Quality',
                'trigger': 'Missing feeds > 10 minutes',
                'action': 'Fallback to stale data, alert'
            }
        ]
        return rules

# Demonstration
production = ProductionNLPSystem()

print("Production NLP System Architecture:")
print("=" * 70)
for layer, components in production.system_architecture().items():
    print(f"\n{layer}:")
    for component, value in components.items():
        print(f"  {component}: {value}")

print("\n" + "=" * 70)
print("\nMonitoring Metrics:")
for category, metrics in production.monitoring_metrics().items():
    print(f"\n{category}:")
    for metric, target in metrics.items():
        print(f"  {metric}: {target}")

print("\n" + "=" * 70)
print("\nFailsafe Rules:")
for rule in production.failsafe_rules():
    print(f"\n  {rule['rule']}:")
    print(f"    Trigger: {rule['trigger']}")
    print(f"    Action: {rule['action']}")

In [ ]:
# Production dashboard visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Latency distribution
np.random.seed(42)
latencies = np.concatenate([
    np.random.exponential(20, 900),  # Normal
    np.random.exponential(100, 100)  # Some slow ones
])

axes[0, 0].hist(latencies, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0, 0].axvline(np.percentile(latencies, 50), color='green', linestyle='--', label=f'p50: {np.percentile(latencies, 50):.0f}ms')
axes[0, 0].axvline(np.percentile(latencies, 99), color='red', linestyle='--', label=f'p99: {np.percentile(latencies, 99):.0f}ms')
axes[0, 0].set_xlabel('Latency (ms)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('NLP Service Latency Distribution')
axes[0, 0].legend()

# Sentiment drift over time
days = np.arange(30)
baseline_sentiment = 0.05
sentiment_drift = baseline_sentiment + 0.02 * np.sin(days/10) + np.random.normal(0, 0.01, 30)
sentiment_drift[20:] += 0.03  # Sudden drift

axes[0, 1].plot(days, sentiment_drift, 'b-', linewidth=2, label='Observed')
axes[0, 1].axhline(baseline_sentiment, color='green', linestyle='--', label='Baseline')
axes[0, 1].axhline(baseline_sentiment + 0.05, color='orange', linestyle=':', label='Warning')
axes[0, 1].axhline(baseline_sentiment + 0.08, color='red', linestyle=':', label='Critical')
axes[0, 1].fill_between(days[20:], baseline_sentiment - 0.1, sentiment_drift[20:], alpha=0.3, color='red')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Average Sentiment')
axes[0, 1].set_title('Sentiment Drift Monitoring')
axes[0, 1].legend(loc='upper left')
axes[0, 1].annotate('Drift Detected!', xy=(22, 0.1), fontsize=10, color='red')

# Signal accuracy over time
accuracy = 0.6 + 0.05 * np.random.randn(30).cumsum() / 10
accuracy = np.clip(accuracy, 0.45, 0.75)

axes[1, 0].plot(days, accuracy, 'g-', linewidth=2)
axes[1, 0].axhline(0.5, color='red', linestyle='--', label='Random (50%)')
axes[1, 0].fill_between(days, 0.5, accuracy, where=accuracy > 0.5, alpha=0.3, color='green')
axes[1, 0].fill_between(days, 0.5, accuracy, where=accuracy <= 0.5, alpha=0.3, color='red')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Signal Accuracy')
axes[1, 0].set_title('Rolling Signal Accuracy (7-day)')
axes[1, 0].legend()
axes[1, 0].set_ylim(0.4, 0.8)

# System health dashboard
health_metrics = {
    'News Feed': 0.99,
    'NLP Service': 0.98,
    'Model Server': 0.97,
    'Signal API': 0.99,
    'Database': 0.999
}

names = list(health_metrics.keys())
values = list(health_metrics.values())
colors = ['green' if v > 0.99 else 'orange' if v > 0.95 else 'red' for v in values]

bars = axes[1, 1].barh(names, values, color=colors)
axes[1, 1].set_xlim(0.9, 1.01)
axes[1, 1].set_xlabel('Uptime')
axes[1, 1].set_title('System Health Dashboard')

for bar, val in zip(bars, values):
    axes[1, 1].text(val + 0.002, bar.get_y() + bar.get_height()/2, 
                    f'{val:.1%}', va='center')

plt.tight_layout()
plt.show()

---

## Week 19 Summary & Key Takeaways

### Topics Mastered:

| Day | Topic | Key Concepts |
|-----|-------|-------------|
| 1 | Text Preprocessing | Tokenization, financial vocab, negation handling |
| 2 | Sentiment Analysis | Lexicon vs ML, Loughran-McDonald |
| 3 | Word Embeddings | Word2Vec, GloVe, domain adaptation |
| 4 | Transformers | BERT, FinBERT, fine-tuning |
| 5 | Topic Modeling | LDA, NMF, coherence validation |
| 6 | Alternative Data | Data sources, alpha evaluation |
| 7 | Interview Review | Production systems, best practices |

### Interview Readiness Checklist:

✅ Can explain text preprocessing for financial documents  
✅ Understand sentiment analysis approaches and trade-offs  
✅ Know difference between static and contextual embeddings  
✅ Can discuss FinBERT architecture and use cases  
✅ Understand topic modeling validation  
✅ Can evaluate alternative data sources  
✅ Know NLP feature engineering for alpha  
✅ Understand production NLP system design  

### Common Interview Mistakes to Avoid:

1. **Using FinBERT for real-time trading** - Too slow, use lexicon or simple ML
2. **Ignoring negations** - "Not good" ≠ "Good" in finance
3. **Using general sentiment lexicons** - Financial language is different
4. **Overfitting to backtest** - Alternative data decays fast
5. **Ignoring latency** - Speed matters for alpha capture

In [ ]:
# Final summary visualization
print("=" * 70)
print("WEEK 19: NLP & ALTERNATIVE DATA - COMPLETE")
print("=" * 70)

skills_matrix = {
    'Text Preprocessing': ['Tokenization', 'Financial vocab', 'Negation handling'],
    'Sentiment Analysis': ['Loughran-McDonald', 'ML approaches', 'FinBERT'],
    'Feature Engineering': ['Sentiment scores', 'Topic features', 'Change metrics'],
    'Production Systems': ['Latency optimization', 'Drift monitoring', 'Circuit breakers'],
    'Alternative Data': ['Source evaluation', 'Alpha decay', 'ROI analysis']
}

print("\n📚 Skills Acquired:")
for category, skills in skills_matrix.items():
    print(f"\n{category}:")
    for skill in skills:
        print(f"  ✓ {skill}")

print("\n" + "=" * 70)
print("\n🎯 Ready for NLP & Alternative Data Interview Questions!")
print("\n" + "=" * 70)